In [4]:
import time

import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import scipy.fftpack
from scipy.signal import argrelextrema

from datetime import datetime as dt

from pylab import rcParams
rcParams['figure.figsize'] = 15, 4

In [5]:
# Define the data we will use.

# Lower case dish names!
data_file_path = '/Volumes/QAG Disk 1/data/moon_09102021'
#data_files = ['5b_x.dat', '5b_y.dat', '1h_x.dat', '1h_y.dat']
dish_names = ['1h', '5b']
antenna_pols = ['x', 'y']

DATA_FILE_EXTENSION = 'dat'

# Dishes 1h and 5b. Their XYZ ECEF coordinates in meters are:
# Lower case dish names!
dishes = [{ 'name' : '1h',
            'x' : '1h_x.dat',
            'y' : '1h_y.dat',
            'XYZ' : [-2524203.0583488033, -4123430.8743299064, 4147685.0269737383] },
          { 'name' : '5b', 
            'x' : '5b_x.dat',
            'y' : '5b_y.dat',
            'XYZ' : [-2523924.2759636184, -4123447.7050980935, 4147836.6033990188] } ]

In [6]:
# Create a list of the schedule
schedule = []

schedule.append([-3,3,'2021-09-10 23:05:00'])
schedule.append([-6,6,'2021-09-10 23:07:00'])
schedule.append([-9,9,'2021-09-10 23:09:00'])
schedule.append([-10,10,'2021-09-10 23:11:00'])
schedule.append([-11,11,'2021-09-10 23:13:00'])
schedule.append([-13,13,'2021-09-10 23:15:00'])
schedule.append([-3,6,'2021-09-10 23:17:00'])
schedule.append([-3,9,'2021-09-10 23:19:00'])
schedule.append([-3,10,'2021-09-10 23:21:00'])
schedule.append([-3,11,'2021-09-10 23:23:00'])
schedule.append([-3,13,'2021-09-10 23:25:00'])
schedule.append([-6,3,'2021-09-10 23:27:00'])
schedule.append([-6,9,'2021-09-10 23:29:00'])
schedule.append([-6,10,'2021-09-10 23:31:00'])
schedule.append([-6,11,'2021-09-10 23:33:00'])
schedule.append([-6,13,'2021-09-10 23:35:00'])
schedule.append([-9,3,'2021-09-10 23:37:00'])
schedule.append([-9,6,'2021-09-10 23:39:00'])
schedule.append([-9,10,'2021-09-10 23:41:00'])
schedule.append([-9,11,'2021-09-10 23:43:00'])
schedule.append([-9,13,'2021-09-10 23:45:00'])
schedule.append([-10,3,'2021-09-10 23:47:00'])
schedule.append([-10,6,'2021-09-10 23:49:00'])
schedule.append([-10,9,'2021-09-10 23:51:00'])
schedule.append([-10,11,'2021-09-10 23:53:00'])
schedule.append([-10,13,'2021-09-10 23:55:00'])
schedule.append([-11,3,'2021-09-10 23:57:00'])
schedule.append([-11,6,'2021-09-10 23:59:00'])
schedule.append([-11,9,'2021-09-11 00:01:00'])
schedule.append([-11,10,'2021-09-11 00:03:00'])
schedule.append([-11,13,'2021-09-11 00:05:00'])
schedule.append([-13,3,'2021-09-11 00:07:00'])
schedule.append([-13,6,'2021-09-11 00:09:00'])
schedule.append([-13,9,'2021-09-11 00:11:00'])
schedule.append([-13,10,'2021-09-11 00:13:00'])
schedule.append([-13,11,'2021-09-11 00:15:00'])
schedule.append([-10,10,'2021-09-11 00:17:00'])
schedule.append([-9,9,'2021-09-11 00:19:00'])
schedule.append([-8,8,'2021-09-11 00:21:00'])

#print(schedule)
print(len(schedule))

39


In [7]:
# Function to create the full path filename for a dish/pol
def get_datafile_fullpath(dish_name, pol):
    """
    Create the full path filename for a dish/pol
    
    Argumants:
        dish_name - name of the dish, like 1h or 5b
        pol       - 'x' or 'y' to specify the polarixation in the filename
        
    Returns:
        The full path filename. None if dish_name is not in the list
    """
    
    if dish_name == None or len(dish_name) != 2:
        return None
    if pol != 'x' and pol != 'y':
        return None
    
    for dish in dishes:
        name = dish.get('name')
        if name == dish_name.lower():
            full_path = f"{data_file_path}/{dish[pol]}"
            return full_path
        
    return None


In [8]:
START_TIME = 1631312409.1568253092447917 #This is UTC 2021-09-10 22:20:09
SAMPLES_PER_SEC = 1920000
BYTES_PER_SAMPLE = 4
SECS_OFFSET = 3

def seek_by_time(file, datetime_str):
    """Find the offset into the file for the given datetime and seek.
    Arguments:
        file - the open file
        datetime_str - A datetime in the form 2021-09-10 22:20:09, this if UTC
    Returns:
        int - the bytes skipped in the file.
    """
    d_start = dt.strptime("2021-09-10 22:20:09", '%Y-%m-%d %H:%M:%S') 
    d = dt.strptime(datetime_str, '%Y-%m-%d %H:%M:%S') 
    diff_secs = int(dt.timestamp(d) - dt.timestamp(d_start)) + SECS_OFFSET

    bytes_to_skip = diff_secs * SAMPLES_PER_SEC * BYTES_PER_SAMPLE
    
    #print(bytes_to_skip)
    
    if file != None:
        file.seek(bytes_to_skip)
    
    return bytes_to_skip

#if seek_by_time(None, "2021-09-10 22:20:10")  != 7680000:
#    print("WARNING: seek_by_time() did not return the correct offset. Please check!")

In [9]:
def readBlock(file, num_samples):
    """ 
    Read the next FFT sized block of 16-bit I/Q data from an opened file.
    
    Arguments:
        file - The reference to the data file, already opened.
        num_samples - The number of 16-bit I/Q samples to read in.
            Example, If you FFT SIZE is 1024, the num_samples should
            be 1024. 2048 16-bit samples (4096 bytes total) will be 
            read and unpacked from the file.
            
        Return - A Numpy array of "num_samples" Complex64 values.
            None if the read is past the end of the file.
    
    """
    # Read the bytes from the file
    data_raw = np.fromfile(file, dtype=np.int16, count=num_samples*2)
    
    # Create a Complex64 Numpy array to hold the values
    data = np.zeros((num_samples,), dtype=np.complex64)
    
    # Reassemble the int16 aray into the Complex64 array.
    # Use a try block to catch the condition where the end of the file
    # has been passed.
    try:
        data[:] = (data_raw[0::2]) + 1j*(data_raw[1::2])
    except:
        return None
    
    return data

In [10]:
def create_waterfall(title1, title2, datetime_str, num_secs, start_chan, end_chan, output_image_name=None):
    
    """Create a waterfall plot
    Arguments:
        filename - The full path filename
        datetime_str - the time offset into the file
        num_secs - number of seconds to process
        start_chan - the start frequency bin
        end_chan - the end frequency bin
        output_image_name- Optional, save an image of the plot
    """
    
    # Perform the FFT and store the result for later plotting.
    # The FFT result will be accumulated in a one dimensional Numpy array
    # of length equal to the FFT_SIZE

    # Define some constants. Here you may wish to change the FFT_SIZE
    FFT_SIZE = 1960000
    CENTER_FREQ_HZ = 1296000000
    BANDWIDTH_HZ   = 1920000
    MHZ = 1048576.0
    NUM_CHANS = end_chan - start_chan
    # How many FFT_SIZE chunks to process. Ideally we would like to
    # process the entire channel, but during code development we may
    # wish to read in a smaller number. Reading in the large data files
    # takes a lot of time.
    NUM = num_secs
    
    fig, axs = plt.subplots(len(dish_names),len(antenna_pols), figsize=(72/8, 60/8))
    
    axis_counter = 0
    
    for dish_name in dish_names:
        for pol in antenna_pols:
            
            file_path = get_datafile_fullpath(dish_name, pol)
            #print(file_path)
            
            # Create an array to hold the data for each horizontal line of the waterfall
            bins = []

            # Open the file
            file = open(file_path, "rb")

            # Seek
            seek_by_time(file, datetime_str)
    
            # Create an array to hold the accumulation for each bin
            #fft_acc = np.zeros((end_chan - start_chan, ))

            # Keep a counter of the number of FFT_SIZE length chunks we processed.
            count = 0

            # Loop through the file. 
            for i in range(NUM):

                #print(i)

                # Read the next block of data from the file
                d = readBlock(file, FFT_SIZE)

                # If None is retruned, this signifies we read to the end of the file.
                # Break out of the loop.
                if d is None:
                    print("d is NONE")
                    break

                yf = scipy.fftpack.fft(d, n=FFT_SIZE, axis=0)

                #Add the results of the FFTp to the fft_acc Numpy array
                #np.add(np.abs(yf[start_chan:end_chan]), fft_acc, out=fft_acc)

                #print(np.abs(yf[start_chan:start_chan+1]), fft_acc[0:1])

                xx = np.abs(yf).tolist()
                yy = scipy.fft.fftshift(xx)
                bins.append(yy[start_chan:end_chan])

            # Draw a plot of the results
            
            col = axis_counter % len(axs)
            row = int(axis_counter / len(axs))
            #print(row, col)

            axs[row, col].imshow(bins, aspect='auto')
    
            axs[row,col].set_title(title2)
    
            axs[row, col].set_title(f'{dish_name} - {pol} [{title1}]', fontsize= 16, fontweight="bold")
            
            if row == 1:
                axs[row, col].set_xlabel('Hz', fontsize=16)
            
            if col == 0:
                axs[row, col].set_ylabel('Seconds', fontsize=16)
            
            axis_counter += 1

    fig.tight_layout()
    #fig.suptitle(title1, fontsize=18)
    #plt.subplots_adjust(top=2.0)
    #plt.title(title1, fontsize=18)

    if output_image_name != None:
        plt.savefig(output_image_name)
        
    #plt.text(-3.5, -3.5, title1, fontsize=24)
        
    # Show!
    #plt.show()
    plt.close(fig)


In [11]:
#create_waterfall("+3/-3 Hz", "  ", "2021-09-10 23:05:00", 60, 1236524, 1236622, None)

In [12]:
print(dishes)
for dish_info in dishes:
    #print(dish_info)
    dish_name = dish_info['name']
    file_path = get_datafile_fullpath(dish_name, 'x')
    file_path = get_datafile_fullpath(dish_name, 'y')
    #create_waterfall(file_path, "3, -3", "  ", "2021-09-10 23:05:00", 60, 1236524, 1236622, None)

[{'name': '1h', 'x': '1h_x.dat', 'y': '1h_y.dat', 'XYZ': [-2524203.0583488033, -4123430.8743299064, 4147685.0269737383]}, {'name': '5b', 'x': '5b_x.dat', 'y': '5b_y.dat', 'XYZ': [-2523924.2759636184, -4123447.7050980935, 4147836.6033990188]}]


In [14]:
for sched in schedule[3:-1]:
title = f"{sched[0]:+d}_{sched[1]:+d} Hz"
image_name = f"images/plots_{sched[0]:+d}_{sched[1]:+d}_hz.png"
print(title, image_name)
create_waterfall(title, "  ", sched[2], 60, 1236524, 1236622, image_name)

-10_+10 Hz images/plots_-10_+10_hz.png
-11_+11 Hz images/plots_-11_+11_hz.png
-13_+13 Hz images/plots_-13_+13_hz.png
-3_+6 Hz images/plots_-3_+6_hz.png
-3_+9 Hz images/plots_-3_+9_hz.png
-3_+10 Hz images/plots_-3_+10_hz.png
-3_+11 Hz images/plots_-3_+11_hz.png
-3_+13 Hz images/plots_-3_+13_hz.png
-6_+3 Hz images/plots_-6_+3_hz.png
-6_+9 Hz images/plots_-6_+9_hz.png
-6_+10 Hz images/plots_-6_+10_hz.png
-6_+11 Hz images/plots_-6_+11_hz.png
-6_+13 Hz images/plots_-6_+13_hz.png
-9_+3 Hz images/plots_-9_+3_hz.png
-9_+6 Hz images/plots_-9_+6_hz.png
-9_+10 Hz images/plots_-9_+10_hz.png
-9_+11 Hz images/plots_-9_+11_hz.png
-9_+13 Hz images/plots_-9_+13_hz.png
-10_+3 Hz images/plots_-10_+3_hz.png
-10_+6 Hz images/plots_-10_+6_hz.png
-10_+9 Hz images/plots_-10_+9_hz.png
-10_+11 Hz images/plots_-10_+11_hz.png
-10_+13 Hz images/plots_-10_+13_hz.png
-11_+3 Hz images/plots_-11_+3_hz.png
-11_+6 Hz images/plots_-11_+6_hz.png
-11_+9 Hz images/plots_-11_+9_hz.png
-11_+10 Hz images/plots_-11_+10_hz.png
-

In [15]:
print("HELLO")

HELLO
